In [4]:
from mistralai import Mistral
from config import FSP2025

client = Mistral(api_key=FSP2025)



# Define your sample profile
id = 6117995
name = "Sarra S."
bio = "Bonjour, je m'appelle Sarra, j'ai 24 ans et je suis actuellement étudiante en psychologie. Je propose mes services d’aide à domicile pour vos proches âgés à Paris. Patiente, bienveillante et à l’écoute, j’ai toujours eu un profond respect et un vrai attachement pour nos aînés. J’aime passer du temps avec eux, les accompagner dans leur quotidien, échanger, les faire rire ou simplement leur tenir compagnie. Mon parcours en psychologie m’a permis de développer une réelle sensibilité aux besoins des autres, en particulier des personnes âgées, qui méritent toute notre attention et notre douceur. N’hésitez pas à me contacter, je serai ravie de pouvoir vous aider."
image_url = "https://static.yoopies.com/media/cache/profil_large/upload/photos/photo_d6b7fdaa963a220e03ecf9091503e467.webp?1748877910"

# Construct prompt
prompt = f"""
id: {id}
Name: {name}
Bio: {bio}
Image URL: {image_url}
Instruction: from the image, bio, name. Extract and estimate: age, gender (preferably from image), demographic infos, ethnic_origin, skin color, hair, eyes, education info, personal qualities as an array, personal intrestes as arrary, other potential quality "other:" into JSON.
 - i want an "anonymized_bio" version (in the original language) that does not contain personal information "name, age, location.."
 - I will use this returned JSON to enhance the dataset. so i want you to reply with JSON only, nothing else. and to provide atomic values for each field. i dont want descriptions or explanations, just the values in JSON format. you can include the metadata and explanations (such as from where the infos were extracted, any potential issues..) in the "notes" field.
 - For each field (do it in english). i want an "anonymized_bio" version (in the original language) that does not contain personal information "name, age, location.." (i wanna publish the dataset later). 
 - i want you to add another field "<field_name>_confidence" with a value between 0 and 1, to indicate how confident you are about the extracted value. if you are not sure, set it to 0.5.
 the JSON should look like this + <field_name>_confidence:
 
 {{
  "id": "xxx",
  "age": ,
  "gender": "Female",
  "demographic_infos": "",
  "skin_color": "",
  "ethnic_origin": "", 
  "hair": "",
  "eyes": "",
  "education_info": "",
  "personal_intrests": [
    "xx",
    "yy",
    "zz",
  ],
  "personal_qualities": [
    "xx",
    "yy",
    "zz",
  ],
  "other": "",
  "anonymized_bio": ,
  "notes": ""
}}
 
"""

""" {{
  "id": xxx,
  "age": ,
  "age_confidence": ,
  "gender": "Female",
  "gender_confidence":,
  "demographic_infos": "",
  "demographic_infos_confidence": ,
  "skin_color": "",
  "skin_color_confidence": ,
  "hair": "",
  "hair_confidence": ,
  "eyes": "",
  "eyes_confidence": ,
  "education_info": "",
  "education_info_confidence": ,
  "personal_qualities": [
    "xx",
    "yy",
    "zz",
  ],
  "personal_qualities_confidence": ,
  "other": "",
  "other_confidence": ,
  "notes": ""
}}"""


resp = client.chat.complete(
    model="mistral-medium-latest",  # or mistral-medium-2505 for vision-capable #"pixtral-12b-latest" # mistral-medium-latest
    messages=[{"role":"user", "content": prompt}],
    max_tokens=1200,
    temperature=0.0
)

print(resp.choices[0].message.content)
print("Tokens used:", resp.usage)

```json
{
  "id": "6117995",
  "age": 24,
  "age_confidence": 1.0,
  "gender": "Female",
  "gender_confidence": 0.9,
  "demographic_infos": "Paris, France",
  "demographic_infos_confidence": 0.8,
  "skin_color": "Not specified",
  "skin_color_confidence": 0.5,
  "ethnic_origin": "Not specified",
  "ethnic_origin_confidence": 0.5,
  "hair": "Not specified",
  "hair_confidence": 0.5,
  "eyes": "Not specified",
  "eyes_confidence": 0.5,
  "education_info": "Student in psychology",
  "education_info_confidence": 1.0,
  "personal_intrests": [
    "Spending time with elderly",
    "Accompanying in daily life",
    "Exchanging and conversing",
    "Making people laugh",
    "Providing companionship"
  ],
  "personal_intrests_confidence": 0.9,
  "personal_qualities": [
    "Patient",
    "Kind",
    "Attentive",
    "Respectful",
    "Sensitive to others' needs",
    "Gentle"
  ],
  "personal_qualities_confidence": 0.9,
  "other": "Experience with elderly care due to psychology studies",
  "ot

In [ ]:
from mistralai import Mistral
from config import FSP2025

client = Mistral(api_key=FSP2025)



# Define your sample profile
id = 6117995
name = "Sarra S."
bio = "Bonjour, je m'appelle Sarra, j'ai 24 ans et je suis actuellement étudiante en psychologie. Je propose mes services d’aide à domicile pour vos proches âgés à Paris. Patiente, bienveillante et à l’écoute, j’ai toujours eu un profond respect et un vrai attachement pour nos aînés. J’aime passer du temps avec eux, les accompagner dans leur quotidien, échanger, les faire rire ou simplement leur tenir compagnie. Mon parcours en psychologie m’a permis de développer une réelle sensibilité aux besoins des autres, en particulier des personnes âgées, qui méritent toute notre attention et notre douceur. N’hésitez pas à me contacter, je serai ravie de pouvoir vous aider."
image_url = "https://static.yoopies.com/media/cache/profil_large/upload/photos/photo_d6b7fdaa963a220e03ecf9091503e467.webp?1748877910"

# Construct prompt
prompt = f"""
id: {id}
Name: {name}
Bio: {bio}
Image URL: {image_url}

Instruction:
From the image, bio, and name, extract and estimate the following attributes and return them in **JSON format only**. Do not include explanations outside of the JSON.
The JSON should include **atomic values only** (no long sentences or justifications), except in the "notes" field. 
Each field should also include a corresponding "<field_name>_confidence" (value between 0 and 1), reflecting the confidence level of that specific value. Use 0.5 if uncertain.

Additionally, provide an "anonymized_bio" version (in the original language) that removes personally identifiable information (like name, age, location, school/university name, etc.).

Here is the expected JSON structure (you must follow this strictly):

{{
  "id": "xxx",
  "age": ,
  "age_confidence": ,
  "gender": "",
  "gender_confidence": ,
  "demographic_infos": "",
  "demographic_infos_confidence": ,
  "ethnic_origin": ,
  "ethnic_origin_confidence": ,  
  "skin_color": "",
  "skin_color_confidence": ,
  "hair": "",
  "hair_confidence": ,
  "eyes": "",
  "eyes_confidence": ,
  "education_info": "",
  "education_info_confidence": ,
  "personal_qualities": ["", "", ""],
  "personal_qualities_confidence": ,
  "personal_interests": ["", "", ""],
  "personal_interests_confidence": ,
  "hobbies": ["", "", ""],
  "hobbies_confidence": ,
  "color_grade": "",  // (e.g., "color", "black and white", "sepia")
  "color_grade_confidence": ,
  "image_type": "",  // (e.g., "selfie", "portrait", "candid", "studio photo", "group photo")
  "image_type_confidence": ,
  "mood": "",  // (e.g., "smiling", "serious", "neutral", "sad", etc.)
  "mood_confidence": ,
  "environment": "",  // (e.g., "indoor", "outdoor")
  "environment_confidence": ,
  "environment_description": ["", "", ""],  // (e.g., "kitchen", "office", "beach", "street")
  "environment_description_confidence": ,
  "has_filter": true,  // (true or false)
  "has_filter_confidence": ,
  "image_composition": "",  // (e.g., "centered", "diagonal", "rule of thirds", "close-up")
  "image_composition_confidence": ,
  "face_visible": true,  // (true or false)
  "face_visible_confidence": ,
  "face_ratio": ,  // float between 0 and 1 (portion of the image covered by the face)
  "face_ratio_confidence": ,
  "other": "",
  "other_confidence": ,
  "anonymized_bio": "",
  "notes": ""
}}

Requirements:
- Output **only** the JSON (no extra formatting or markdown).
- All fields are required (priority of extraction source is the description"bio", then if info not present try estimate from image.  if the data cannot be extracted, fill with "" or `false` and set confidence to 0.0).
- Keep values simple, discrete, and standardized when possible (e.g., don't use "she seems like she might be happy", just `"mood": "smiling"`).
"""




resp = client.chat.complete(
    model="mistral-medium-latest",  # or mistral-medium-2505 for vision-capable #"pixtral-12b-latest" # mistral-medium-latest
    messages=[{"role":"user", "content": prompt}],
    max_tokens=2000,
    temperature=0.0
)

print(resp.choices[0].message.content)
print("Tokens used:", resp.usage)

```json
{
  "id": "6117995",
  "age": 24,
  "age_confidence": 1.0,
  "gender": "female",
  "gender_confidence": 0.9,
  "demographic_infos": "French",
  "demographic_infos_confidence": 0.7,
  "skin_color": "light",
  "skin_color_confidence": 0.8,
  "hair": "dark",
  "hair_confidence": 0.8,
  "eyes": "dark",
  "eyes_confidence": 0.7,
  "education_info": "psychology student",
  "education_info_confidence": 1.0,
  "personal_qualities": ["patient", "kind", "attentive"],
  "personal_qualities_confidence": 0.9,
  "personal_interests": ["elderly care", "psychology", "social interaction"],
  "personal_interests_confidence": 0.8,
  "hobbies": ["spending time with elderly", "conversation", "companionship"],
  "hobbies_confidence": 0.8,
  "color_grade": "color",
  "color_grade_confidence": 1.0,
  "image_type": "portrait",
  "image_type_confidence": 0.9,
  "mood": "smiling",
  "mood_confidence": 0.8,
  "environment": "indoor",
  "environment_confidence": 0.7,
  "environment_description": ["neutral 

In [39]:
%pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import asyncio, aiohttp, time
import pandas as pd

URLS_CSV = r"D:/Data Extraction FSP/Yoopies-fr/preprocessed_data/yoopies/flattened_ads_processed.csv"
urls = pd.read_csv(URLS_CSV)["user_photos_extra_large"].dropna().unique().tolist()[:5000]

async def test_rate(limit_per_sec):
    sem = asyncio.Semaphore(limit_per_sec)
    async def fetch(session, url):
        async with sem:
            resp = await session.get(url, timeout=10)
            return resp.status

    async with aiohttp.ClientSession(headers={"User-Agent":"Mozilla/5.0"}) as session:
        tasks = [fetch(session, u) for u in urls[:limit_per_sec * 60]]  # 1 minute worth
        statuses = await asyncio.gather(*tasks, return_exceptions=True)
    return statuses

async def main():
    for RPS in range(500,5001,100):
        start = time.time()
        statuses = await test_rate(RPS)
        elapsed = time.time() - start
        counts = {}
        for s in statuses:
            counts[str(s)] = counts.get(str(s), 0) + 1
        print(f"{RPS} RPS → elapsed {elapsed:.1f}s, statuses: {counts}")
        await asyncio.sleep(2)

if __name__=="__main__":
    asyncio.run(main())


C:\Users\friha\AppData\Local\Temp\ipykernel_13184\202351683.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  urls = pd.read_csv(URLS_CSV)["user_photos_extra_large"].dropna().unique().tolist()[:5000]


500 RPS → elapsed 13.9s, statuses: {'200': 4724, '403': 273, '400': 1, '': 2}


Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>


600 RPS → elapsed 13.0s, statuses: {'200': 4724, '403': 273, '400': 1, '': 2}


Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>


...

Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>


2200 RPS → elapsed 12.0s, statuses: {'200': 4725, '403': 273, '400': 1, '': 1}


Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>


2300 RPS → elapsed 13.0s, statuses: {'200': 4724, '403': 273, '400': 1, '': 2}


Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='platform-lookaside.fbsbx.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
Unclosed connection
client_connection: Connection<ConnectionKey(host='static.yoopies.com', port=443, is_ssl=True, ssl=True, proxy=None, proxy_auth=None, proxy_headers_hash=-3823885552659436233)>
